In [59]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from prophet import Prophet
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
import numpy as np
import plotly.io as pio


In [60]:
# Load the data
file_path = '/content/Strategic_Measure_Dollar_Amount_and_Percentage_Increase_of_Major_Rates_and_Fees.csv'
data = pd.read_csv(file_path)
data.head()

,Fiscal Year,Fiscal Year Start Date,Service or Fee,Approved Rate,Actual Rate,Annual Dollar Change from Previous Fiscal Year,Percent Change from Previous Fiscal Year,Typical Ratepayer Definition
0,2022,10/01/2021 12:00:00 AM,Austin Energy,1015.20,1015.20,0.00,0.000,Residential customer usage of 860 Kwh.
1,2022,10/01/2021 12:00:00 AM,Austin Water,969.48,969.48,0.00,0.000,"Residential customer usage of 5,800 gallons of..."
2,2022,10/01/2021 12:00:00 AM,Austin Resource Recovery,348.60,330.60,18.00,0.054,Residential customer using a 64-gallon cart
3,2022,10/01/2021 12:00:00 AM,Clean Community Fee,111.60,107.40,4.20,0.039,Per single-family home
4,2022,10/01/2021 12:00:00 AM,Transportation User Fee,179.52,156.48,23.04,0.147,Per single-family home


In [58]:
#Convert 'Fiscal Year Start Date' to datetime
print(data['Fiscal Year Start Date'].unique())

['10/01/2021 12:00:00 AM' '10/01/2020 12:00:00 AM'
 '10/01/2019 12:00:00 AM' '10/01/2018 12:00:00 AM'
 '10/01/2017 12:00:00 AM' '10/01/2016 12:00:00 AM'
 '10/01/2015 12:00:00 AM' '10/01/2014 12:00:00 AM'
 '10/01/2013 12:00:00 AM']


In [57]:
 #Check for missing values
print("Missing values per column:\n", data.isnull().sum())

Missing values per column:
 Fiscal Year                                       0
Fiscal Year Start Date                            0
Service or Fee                                    0
Approved Rate                                     0
Actual Rate                                       0
Annual Dollar Change from Previous Fiscal Year    0
Percent Change from Previous Fiscal Year          0
Typical Ratepayer Definition                      0
dtype: int64


In [61]:
# --- EDA ---

In [62]:
# Plot: Distribution of Actual Rate
fig1 = px.histogram(data, x='Actual Rate', nbins=30, title='Distribution of Actual Rates', color_discrete_sequence=['#636EFA'])
fig1.show()


In [63]:
# Plot: Actual Rate over Time
fig2 = px.line(data, x='Fiscal Year Start Date', y='Actual Rate', color='Service or Fee', title='Actual Rate Trend Over Time')
fig2.show()

In [64]:
# Plot: Top 10 services by average Actual Rate
top_services = data.groupby('Service or Fee')['Actual Rate'].mean().sort_values(ascending=False).head(10)
fig3 = px.bar(top_services, x=top_services.index, y=top_services.values, title='Top 10 Services by Average Actual Rate', labels={'x':'Service or Fee', 'y':'Average Actual Rate'}, color_discrete_sequence=['#EF553B'])
fig3.update_layout(xaxis_tickangle=-45)
fig3.show()

In [65]:
# Plot: Correlation Heatmap
corr = data[['Approved Rate', 'Actual Rate', 'Annual Dollar Change from Previous Fiscal Year', 'Percent Change from Previous Fiscal Year']].corr()
fig4 = px.imshow(corr, text_auto=True, title='Correlation Heatmap')
fig4.show()

In [66]:
# --- Time Series Forecasting with Prophet ---


In [67]:
#Ensure correct data types
numeric_columns = ['Approved Rate', 'Actual Rate',
                   'Annual Dollar Change from Previous Fiscal Year',
                   'Percent Change from Previous Fiscal Year']


In [68]:
for col in numeric_columns:
    data[col] = pd.to_numeric(data[col], errors='coerce')

In [69]:
#Quick look at the cleaned data
print("\nCleaned Data Overview:\n", data.head())


Cleaned Data Overview:
    Fiscal Year  Fiscal Year Start Date            Service or Fee  \
0         2022  10/01/2021 12:00:00 AM             Austin Energy   
1         2022  10/01/2021 12:00:00 AM              Austin Water   
2         2022  10/01/2021 12:00:00 AM  Austin Resource Recovery   
3         2022  10/01/2021 12:00:00 AM       Clean Community Fee   
4         2022  10/01/2021 12:00:00 AM   Transportation User Fee   

   Approved Rate  Actual Rate  Annual Dollar Change from Previous Fiscal Year  \
0        1015.20      1015.20                                            0.00   
1         969.48       969.48                                            0.00   
2         348.60       330.60                                           18.00   
3         111.60       107.40                                            4.20   
4         179.52       156.48                                           23.04   

   Percent Change from Previous Fiscal Year  \
0                               

In [70]:
# Forecasting function
forecast_dict = {}

def forecast_service(service_name):
    service_data = data[data['Service or Fee'] == service_name][['Fiscal Year Start Date', 'Actual Rate']]
    service_data = service_data.rename(columns={'Fiscal Year Start Date': 'ds', 'Actual Rate': 'y'})

    model = Prophet(n_changepoints=6 )
    model.fit(service_data)

    future = model.make_future_dataframe(periods=3, freq='YE')
    forecast = model.predict(future)
    return forecast, service_data # Return forecast and service_data


In [71]:
# --- Cell for model evaluation ---
services = data['Service or Fee'].unique()
forecast_dict = {} # Reinitialize the dictionary to store all forecasts

for service in services:
    forecast, service_data = forecast_service(service)
    # Evaluate model (only on historical)
    forecast_trimmed = forecast.iloc[:len(service_data)]
    mape = mean_absolute_percentage_error(service_data['y'], forecast_trimmed['yhat'])
    rmse = np.sqrt(mean_squared_error(service_data['y'], forecast_trimmed['yhat']))
    forecast_dict[service] = (forecast, mape, rmse) # Use 'service' as the key

/usr/local/lib/python3.11/dist-packages/prophet/forecaster.py:1133: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/usr/local/lib/python3.11/dist-packages/prophet/forecaster.py:287: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmprd8jwx8c/awa_6gx7.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmprd8jwx8c/5c0v_kax.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/p

In [72]:
top_3_services = data['Service or Fee'].value_counts().head(3).index.tolist()

fig5 = px.line(
    data[data['Service or Fee'].isin(top_3_services)],
    x='Fiscal Year Start Date',
    y='Actual Rate',
    color='Service or Fee',
    title='Time Series of Actual Rates for Top 3 Services',
    labels={'Fiscal Year Start Date': 'Year', 'Actual Rate': 'Rate'},
    line_shape='linear',
    markers=True
)
fig5.update_layout(
    plot_bgcolor='white',
    paper_bgcolor='#f9f9f9',
    title_font_size=20,
    font=dict(color='#333333')
)
fig5.show()


In [73]:
pip install dash plotly pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 13.5 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [79]:
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output # This import remains the same

In [81]:
app = dash.Dash(__name__)
app.layout = html.Div(style={'backgroundColor': '#f9f9f9', 'padding': '20px'}, children=[
    html.H1('📈 Service Rate Forecast Dashboard', style={'textAlign': 'center', 'color': '#003366'}),

    html.Div([
        html.Label('Select a Service:', style={'fontWeight': 'bold', 'marginBottom': '10px'}),
        dcc.Dropdown(
            id='service-dropdown',
            options=[{'label': service, 'value': service} for service in services],
            value=services[0],
            style={'width': '60%', 'margin': 'auto'}
        ),
    ], style={'textAlign': 'center', 'marginBottom': '30px'}),

    html.Div([
        dcc.Graph(id='forecast-graph', config={'displayModeBar': False}),
    ], style={'backgroundColor': 'white', 'padding': '20px', 'borderRadius': '15px', 'boxShadow': '0px 4px 8px rgba(0, 0, 0, 0.1)'}),

    html.Div(id='metrics', style={
        'marginTop': '20px',
        'textAlign': 'center',
        'fontSize': '20px',
        'color': '#006400',
        'fontWeight': 'bold'
    })
])

@app.callback(
    [Output('forecast-graph', 'figure'),
     Output('metrics', 'children')],
    [Input('service-dropdown', 'value')]
)
def update_graph(selected_service):
    forecast, mape, rmse = forecast_dict[selected_service]

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], mode='lines', name='Predicted', line=dict(color='#1f77b4')))
    fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat_upper'], mode='lines', name='Upper Bound', line=dict(dash='dot', color='lightblue')))
    fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat_lower'], mode='lines', name='Lower Bound', line=dict(dash='dot', color='lightblue')))

    fig.update_layout(
        title=f'Forecast for {selected_service}',
        xaxis_title='Year',
        yaxis_title='Actual Rate',
        plot_bgcolor='white',
        paper_bgcolor='#f9f9f9',
        font=dict(color='#333333')
    )

    metrics_text = f"✅ MAPE: {mape:.2%} | RMSE: {rmse:.2f}"

    return fig, metrics_text

if __name__ == '__main__':
    app.run(debug=False)




<IPython.core.display.Javascript object>